# Karateclub algorithms

In [1]:
from karateclub import EgoNetSplitter
from karateclub import LabelPropagation
from karateclub import DANMF
from karateclub import GEMSEC
import networkx as nx
import pickle

import numpy as np

G = nx.read_edgelist("com-youtube-connected")

In [2]:
file_indexes = [index for index in range(G.number_of_nodes())]
graph_indexes = [i for i in range(len(file_indexes))]
file_index_to_graph_index = dict()
graph_index_to_file_index = dict()
for i in range(len(file_indexes)):
    file_index_to_graph_index[file_indexes[i]] = graph_indexes[i]
    graph_index_to_file_index[graph_indexes[i]] = file_indexes[i]
G = nx.convert_node_labels_to_integers(G)

# Ego-splitting

In [4]:
splitter = EgoNetSplitter(1.0)
# OOM
splitter.fit(G)
print(splitter.get_memberships())

# Label propagation

In [3]:
model = LabelPropagation()
model.fit(G)
cluster_membership = model.get_memberships()

In [5]:
with open('LabelPropagationPredictions.pkl', 'wb') as f:
    pickle.dump(cluster_membership, f)

# DANMF

In [3]:
model = DANMF()
# OOM
model.fit(G)
cluster_membership = model.get_memberships()

/home/geraud/Machine-Learning-in-Network-Science/.venv/lib/python3.8/site-packages/karateclub/community_detection/overlapping/danmf.py:48: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  self._A = nx.adjacency_matrix(
<class 'networkx.utils.decorators.argmap'> compilation 13:4: FutureWarning: laplacian_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
/home/geraud/Machine-Learning-in-Network-Science/.venv/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 100 reached. Increase it to improve convergence.
  warnings.warn(


MemoryError: Unable to allocate 18.8 GiB for an array with shape (2521454096,) and data type int64

In [ ]:
import pickle

In [ ]:
with open('DANMFPredictions.pkl', 'wb') as f:
    pickle.dump(cluster_membership, f)

# GEMSEC

In [3]:
model = GEMSEC()
# OOM
model.fit(G)
cluster_membership = model.get_memberships()

In [ ]:
with open('GEMSECPredictions.pkl', 'wb') as f:
    pickle.dump(cluster_membership, f)

# Evaluation

from sklearn.metrics.cluster import normalized_mutual_info_score

with open('LabelPropagationPredictions.pkl', 'rb') as f:
    cluster_membership = pickle.load(f)

cluster_membership = [cluster_membership[node] for node in range(len(cluster_membership))]

nmi = normalized_mutual_info_score(target, cluster_membership)
print('NMI: {:.4f}'.format(nmi))